In [49]:
print(f"ELK Upgrade Index mapping compare")

ELK Upgrade Index mapping compare


In [50]:
!pip install jsondiff


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
#!pip install ipywidgets

In [52]:
from elasticsearch import Elasticsearch
import os
import json
import pandas as pd
import jsondiff
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

In [53]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables 

True

In [54]:
from ipywidgets import interact, widgets
from IPython.display import display

In [55]:
from ipywidgets import interact

# @interact
# def greet(name="World", count=5):
#     for _ in range(count):
#         print(f"Hello, {name}!")


In [56]:
''' Input ES url '''
# print('Enter source ES cluster url:')
# source = input()
# print('Enter target ES cluster url:')
# target = input()
source = "http://localhost:9201"
target = "http://localhost:9202"


In [57]:
def get_headers():
    ''' Elasticsearch Header '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : '{}'.format(os.getenv('BASIC_AUTH')),
            # 'Connection': 'close'
    }

In [58]:
def get_es_instance(host):
    es_client = Elasticsearch(hosts="{}".format(host), headers=get_headers(), timeout=5,  verify_certs=False)
    return es_client

''' Source cluster '''
es_obj_s_client = get_es_instance(f"{source}")
''' Target cluster '''
es_obj_t_client = get_es_instance(f"{target}")

In [59]:
resp = es_obj_s_client.cluster.health()
print(json.dumps(resp, indent=2))

{
  "cluster_name": "docker-cluster",
  "status": "yellow",
  "timed_out": false,
  "number_of_nodes": 1,
  "number_of_data_nodes": 1,
  "active_primary_shards": 114,
  "active_shards": 114,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 114,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 50.0
}


In [60]:
df = pd.DataFrame.from_dict([resp])
df.head(10)

,cluster_name,status,timed_out,number_of_nodes,number_of_data_nodes,active_primary_shards,active_shards,relocating_shards,initializing_shards,unassigned_shards,delayed_unassigned_shards,number_of_pending_tasks,number_of_in_flight_fetch,task_max_waiting_in_queue_millis,active_shards_percent_as_number
0,docker-cluster,yellow,False,1,1,114,114,0,0,114,0,0,0,0,50.0


In [61]:
resp = es_obj_t_client.cluster.health()
print(json.dumps(resp, indent=2))

{
  "cluster_name": "docker-elasticsearch",
  "status": "yellow",
  "timed_out": false,
  "number_of_nodes": 1,
  "number_of_data_nodes": 1,
  "active_primary_shards": 80,
  "active_shards": 80,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 56,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 58.82352941176471
}


## Compare the mapping from all indices in source cluster with target cluster

In [62]:
from rich.markdown import Markdown
MARKDOWN = f"""
Reindex Report 
between source cluster : {es_obj_s_client} and target Cluster : {es_obj_t_client}
"""
Markdown(MARKDOWN)

Reindex Report between source cluster : <Elasticsearch([{'host': 'localhost', 'port': 9201}])> and target Cluster :
<Elasticsearch([{'host': 'localhost', 'port': 9202}])>

In [63]:
# source_idx_lists = ['om_whorder_02072022_22_2_1']
source_idx_lists = list(es_obj_s_client.indices.get("*"))
# print(source_idx_lists)

In [64]:
response = {}
all_same_mapping = []

In [65]:
def compare_mapping(index_name, diff):
    ''' compare diff using jsondiff library '''
    if not diff:
        all_same_mapping.append(True)
        response.update({index_name : {'diff' : 'Same mapping'}})
    else:
        all_same_mapping.append(False)
        response.update({index_name : {'diff' : 'Different mapping', 'result' : diff}})
    return response

In [66]:
def lookup_type_in_indices(key):
    ''' lookup type we want to compare from the source es cluster '''
    if "OM_" in key or "WX_" in key or "ES_" in key or "ARCHIVE_" in key:
        return True
    return False

In [67]:
def get_mapping_from_properties(mapping, es_v5=False):
    if es_v5:
        return {"properties" : v2.get("properties") for k, v in mapping.items() for k1, v1 in v.items() for k2, v2 in v1.items() if lookup_type_in_indices(k2)} 
    else:
        return {'properties': v2 for k, v in mapping.items() for k1, v1 in v.items() for k2, v2 in v1.items() }
            

In [68]:
def es_version_verify(es_client):
    # print(es_client.info()['version']['number'], type(es_client.info()['version']['number']))
    ''' if es_client v.5.X '''
    if "5." in es_client.info()['version']['number']:
        return True
    else:
        return False

In [69]:
for index_name in source_idx_lists:
    ''' real index '''
    if index_name.startswith("wx_") or index_name.startswith("om_") or index_name.startswith("es_") or index_name.startswith("archive_es_"):
        
        try:
            source_mapping = es_obj_s_client.indices.get_mapping(index=index_name)
            target_mapping = es_obj_t_client.indices.get_mapping(index=index_name)
        
            ''' Get ES v.5.6.4 mapping '''
            source_mapping = get_mapping_from_properties(source_mapping, es_v5=es_version_verify(es_obj_s_client))
            # print(source_mapping)

            ''' Get ES v.8.17 mapping '''
            target_mapping = get_mapping_from_properties(target_mapping, es_v5=es_version_verify(es_obj_t_client))
            # print(target_mapping)
        
            # Compare JSON objects using jsondiff
            # diff = jsondiff.diff(source_mapping, target_mapping, marshal=True, syntax="symmetric")
            diff = jsondiff.diff(source_mapping, target_mapping, marshal=True)
        
            ''' Compare mapping the specific index_name between source/target cluster '''
            compare_mapping(index_name, diff)

        except Exception as e:
            print(e)


NotFoundError(404, 'index_not_found_exception', 'no such index [es_pipeline_upload_test03152018_01]', es_pipeline_upload_test03152018_01, index_or_alias)


In [70]:
from rich.markdown import Markdown
MARKDOWN = f"""
Reindex Report 
between source cluster : {es_obj_s_client} and target Cluster : {es_obj_t_client}
"""
Markdown(MARKDOWN)

Reindex Report between source cluster : <Elasticsearch([{'host': 'localhost', 'port': 9201}])> and target Cluster :
<Elasticsearch([{'host': 'localhost', 'port': 9202}])>

In [71]:
''' Compare mapping the specific index_name between source/target cluster '''
print(json.dumps(response, indent=2))

{
  "archive_es_del_queue_active_test": {
    "diff": "Same mapping"
  },
  "es_pipeline_upload_test_om": {
    "diff": "Same mapping"
  },
  "es_pipeline_upload_test_wm": {
    "diff": "Same mapping"
  }
}


In [72]:
is_same_mapping =  all(all_same_mapping) if all_same_mapping else api_is_same_mapping

In [73]:
print('**')
print(f"Same mappings for all indices between {es_obj_s_client} and {es_obj_t_client}?")
if response:
    print(f"Same Mapping? {is_same_mapping}")
print('**')
print('**')

**
Same mappings for all indices between <Elasticsearch([{'host': 'localhost', 'port': 9201}])> and <Elasticsearch([{'host': 'localhost', 'port': 9202}])>?
Same Mapping? True
**
**
